In [1]:
!pip install selenium
!pip install pandas
!pip install chromedriver-autoinstaller
!pip install BeautifulSoup4


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests #manggil librari buat ke web
import pandas as pd #buat ngolah data
from selenium import webdriver #buat hubungin ke chrome

In [3]:
import time
import re

In [4]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup  # Make sure to import BeautifulSoup

# Set up the Selenium web driver (provide the path to your ChromeDriver)
driver_path = r'C:\Users\Seprian Najib Hilmi\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

Data = []
i = 1

for page in range(1):
    # URL of the news website you want to scrape
    url = "https://www.tribunnews.com/search?q=Pencemaran+Kalimantan+Timur+&cx=partner-pub-7486139053367666%3A4965051114&cof=FORID%3A10&ie=UTF-8&siteurl=www.tribunnews.com#gsc.tab=0&gsc.q=Pencemaran%20Kalimantan%20Timur%20&gsc.page=" + str(page)

    # Send an HTTP GET request to the website using Selenium
    driver.get(url)

    # Wait for a few seconds to let the page load (you can increase this if needed)
    time.sleep(5)

    # Now you can access the fully loaded HTML content
    page_source = driver.page_source

    # Parse the web page content using BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")

    # Find and extract news headlines and their URLs
    beritas = soup.find_all('div', class_='gsc-webResult gsc-result')

    for berita in beritas :
        judulBeritaElement = berita.find('div', class_='gs-title')
        if judulBeritaElement:
            judulBerita = judulBeritaElement.text.strip()
            #print(judulBerita)
                
        dataBeritaElement = berita.find('div', class_='gsc-table-cell-snippet-close')
        if dataBeritaElement:
            dataBerita = dataBeritaElement.text.strip()
            #Define a regular expression pattern to match the date
            date_pattern = r'(\w+ \d{1,2}, \d{4})'

            #Use re.search to find the date in the 'Data' field
            date_match = re.search(date_pattern, dataBerita)

            if date_match:
                date = date_match.group(0)
            
        link = berita.find('a')['href']
        
        if '...' in judulBerita:
            try:
                driver.get(link)
                time.sleep(3)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source, "html.parser")
                judul = soup.find('h1', class_='f50 black2 f400 crimson')
                
                judulBerita = judul.text.strip()
            except Exception as e:
                print("Gagal mendapatkan judul:", e)
               
            for isi in isiBeritas :
                    konten = soup.find('div', class_='side-article txt-article multi-fontsize editcontent')
                    kontenBerita = konten.text.strip()
                
        
        Data.append({'Nomor': i, 'Judul': judulBerita, 'Tanggal': dataBerita, 'Link': link})
            #'Link': link
#         , 'tanggal dan sumber': tanggalBerita, 'Isi Singkat': kontenBerita
        i+=1
        print(Data[-1])

    




{'Nomor': 1, 'Judul': 'Pencemaran Air di Kelurahan Sangasanga Dalam Kaltim Tunjukan Lemahnya Pengelolaan Limbah Perusahaan', 'Tanggal': 'Pencemaran Air di Kelurahan Sangasanga Dalam Kaltim Tunjukan ...Apr 8, 2021 ... TRIBUNNEWS.COM, TENGGARONG - Jaringan Advokasi Tambang Kalimantan Timur (Jatam Kaltim) angkat bicara terkait pencemaran air yang diduga limbah\xa0...www.tribunnews.comhttps://www.tribunnews.com/.../pencemaran-air-di-kelurahan-sangasanga-d...', 'Link': 'https://www.tribunnews.com/regional/2021/04/08/pencemaran-air-di-kelurahan-sangasanga-dalam-kaltim-tunjukan-lemahnya-pengelolaan-limbah-perusahaan'}
{'Nomor': 2, 'Judul': 'Sebabkan Pencemaran, Masyarakat Tuntut PT Saraswanti Lakukan Pemulihan Fungsi Lingkungan', 'Tanggal': 'Sebabkan Pencemaran, Masyarakat Tuntut PT Saraswanti Lakukan ...Sep 7, 2023 ... Sumber: Tribun Kaltim · pencemaran lingkungan · Limbah Industri · Kabupaten Paser · perusahaan sawit · TribunKaltim.co.kaltim.tribunnews.comhttps://kaltim.tribunnews.com/.../s

In [15]:
df = pd.DataFrame(Data)
df.to_csv('PencemaranKalimantanTimur.csv', index=False)